In [ ]:
# This cell is used for creating a button that hides/unhides code cells to quickly look only the results.
# Works only with Jupyter Notebooks.

from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
# Description:
#   Exercise5 notebook.
#
# Copyright (C) 2018 Santiago Cortes, Juha Ylioinas
#
# This software is distributed under the GNU General Public 
# Licence (version 2 or later); please refer to the file 
# Licence.txt, included with the software, for details.

# Preparations
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Select data directory
if os.path.isdir('/coursedata'):
    # JupyterHub
    course_data_dir = '/coursedata'
elif os.path.isdir('../../../coursedata'):
    # Local installation
    course_data_dir = '../../../coursedata'
else:
    # Docker
    course_data_dir = '/home/jovyan/work/coursedata/'

print('The data directory is %s' % course_data_dir)
data_dir = os.path.join(course_data_dir, 'exercise-05-data/')
print('Data stored in %s' % data_dir)

# CS-E4850 Computer Vision Exercise Round 5

The exercises should be solved and the solutions submitted via Aalto JupyterHub by the deadline. 

**Deliverables:**
- **Jupyter notebook** (.ipynb) containing your solutions to the programming tasks. Do not change the name of the notebook file. It may result in 0 points for the exercise.
- **PDF** with your solutions to any pen-and-paper tasks (scanned or typeset, e.g., LaTeX). Make sure the PDF is clearly readable.

**Important:**
- Fill only the cells marked with `# YOUR CODE HERE`. Do not change function signatures.
- You may add extra cells for your own tests, but **do not** overwrite global variables or edit locked cells.
- **Never create new cells by menu commands "Edit/Copy Cells" and "Edit/Paste Cells ..."**. These commands create cells with duplicate ids and make autograding impossible. Use menu commands "Insert/Insert Cell ..." or the button with a plus sign to insert new cells.
- **All notebooks contain hidden tests** which are used for grading. They are hidden inside read-only cells. Therefore, **the read-only cells should never be removed.** 
- **Note:** Visible tests mainly check the shapes and data types of your function’s output. Hidden tests check the correctness of your solution more thoroughly. Passing the visible tests does not guarantee full points for the exercise.
- **Google Colab warning:** Uploading your assignment notebooks to Colab may cause problems. Colab can overwrite notebook metadata and break the autograding. To avoid this, we recommend copy-pasting your code into the notebooks fetched on JupyterHub. Sorry for the inconvenience.
- Be sure that everything that you need to implement should work with the pictures specified by the assignments of this exercise round.
- Running the cells in mixed order (which quite often happens while trying different things and debugging) may cause errors. While working on a particular cell be sure that you have freshly run all its preceding cells belonging to the same exercise.
- **Before submitting**, simply run all the cells of the notebook (for example, select "Restart & Run All" in the menu) and check that all the cells run properly.
- **Remember to submit your assignment!**

## Remember to do the pen-and-paper assignments given in Exercise05.pdf.


## Exercise 1. - Total least squares line fitting. (Pen and paper problem, 10 points)

This is a pen-and-paper exercise, which is given in Exercise05.pdf. Submit your solutions for this exercise in PDF format. 

## Exercise 2. - Robust line fitting using RANSAC (10 points)

In this exercise, you will implement robust line fitting using the RANSAC algorithm.

### Steps:

- **2.1** Load and plot data points
- **2.2** Find a line that minimizes the total least squares fitting
- **2.3** Implement RANSAC line fitting
- **2.4** Display the RANSAC fit and inliers

Load and visualize the data in **2.1**, complete the tasks in **2.2** and **2.3**, and visualize the results in **2.4**.


### 2.1 Load and plot data points

In [ ]:
data = np.load(data_dir+'points.npy')
x, y = data[0,:], data[1,:]
plt.plot(x, y, 'kx')
plt.title('Point distribution')
plt.axis('equal')
plt.show()

### 2.2 Find line that minimizes the total least squares fitting

The final step of robust line fitting using RANSAC involves fitting a line to all inlier points using **total least squares**. In this task, you will implement total least squares fitting so it can be used later in your RANSAC algorithm (Step 2.3).

### Task:
Implement `linefitlsq(x, y)` function that applies total least squares line fitting to the given points and returns the coefficients $(a,b,d)$ of the fitted line  $a x + b y - d = 0$, with $a^2 + b^2 = 1$ (scale is arbitrary; any proportional vector defines the same line).

### Function specification:
- **Input:** `x` and `y` as 1D NumPy arrays of equal length  
- **Output:** a NumPy array `l = np.array([a, b, d])` representing the line $a x + b y - d = 0$  

### Hint: 
- Review Exercise 1 and the slides of Lecture 4
- You may use: `np.mean`, `np.vstack`, `np.linalg.eig`.


In [ ]:
def linefitlsq(x,y):
    ##--your-code-starts-here--##
    # YOUR CODE HERE
    raise NotImplementedError()
    ##--your-code-ends-here--##
    l = np.array([a,b,d])
    
    return l

In [ ]:
# Visible tests

l = linefitlsq(x, y)

# Existence, type and shape
assert isinstance(l, np.ndarray) and l.shape == (3,), "Return value must be np.ndarray of shape (3,)"

# Create synthetic data: y = 0.5x + 1
rng = np.random.default_rng(0)
x = np.linspace(-5, 5, 200)
y_true = 0.5 * x + 1.0
y = y_true + rng.normal(0, 0.05, size=x.shape)

l = linefitlsq(x, y)

assert abs(0.5 + l[0] / l[1]) < 0.05, "Slope is incorrect."
assert abs(1.0 - l[2] / l[1]) < 0.05, "Intercept is incorrect."

print("All visible tests passed.")

In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


### 2.3 RANSAC

#### Task:
Implement a **RANSAC approach** to estimate a line that best fits to given points as explained in the **slides of Lecture 4**.

Repeat the following steps until the required number of iterations `N` is reached.  
Start with `N` set to a large value, and update it **adaptively** during the process using the best inlier ratio found so far (see lecture for details):

- Randomly select 2 distinct points $(x_i, y_i)$. (If the same point is drawn twice, redraw.)
- Fit a line to these 2 points.
- Determine the inliers to this line among the remaining points (i.e. points whose distance to the line is less than the given threshold `t`).

After reaching the required number of iterations, take the line with the highest number of inliers from the previous stage and **refit** it using **total least squares** on the set of points identified as inliers to that line.

#### Return:
- The estimated coefficients $(a,b,d)$  of the fitted line  $a x + b y - d = 0$, with $a^2 + b^2 = 1$ (as in Step 2.2) and 
- The indeces of the inlier points

#### Important:
- All required RANSAC parameters (threshold `t`, probability `p`, etc.) are already given at the beginning of the algorithm.
- Update `max_inliers` (the largest inlier count observed so far). In this task you will **implement the adaptive variant** of `ransac_line_fit`, which **adaptively determine the number of needed iterations**, as presented in the lecture slides. The adaptive logic is already implemented; your task is to keep `max_inliers` up to date.

In [ ]:
def ransac_line_fit(x, y):

    # m is the number of data points
    m = np.size(x) * 1.0
    # s is the size of the random sample
    s = 2
    # t is the inlier distance threshold
    t = np.sqrt(3.84) * 2
    # e is the expected outlier ratio
    e = 0.8
    # at least one random sample should be free 
    # from outliers with probability p
    p = 0.999
    # required number of samples
    N_estimated = np.log(1-p) / np.log(1-(1-e)**s)
    
    # First initialize some variables
    N = np.inf
    sample_count = 0
    max_inliers = 0
    best_line = np.zeros((3,1))
    
    while N > sample_count:
        
        ##--your-code-starts-here--##
        # Pick two random samples
        # if the same point is drawn twice, pick again
        
        # Fit line to these two points
        
        # Determine the inliers of this line
        # (i.e. points whose distance to the line is less than the given threshold t)
        
        # Keep the line giving most inliers so far
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Update the outlier ratio estimate based on the best model so far.
        # max_inliers = highest number of inliers found up to this iteration
        #max_inliers
        ##--your-code-ends-here--## 
        
        e = 1 - float(max_inliers) / float(m)

        # Update also the estimate for the required number of samples
        N = np.log(1-p)/np.log(1-(1-e)**s)
        sample_count += 1
        
    
    # Perform total least squares fitting on the inliers of the best RANSAC hypothesis:
    # 1. Recompute the inliers for the best line found
    # 2. Refit the line using total least squares on all these inliers
    ##--your-code-starts-here--##
    # YOUR CODE HERE
    raise NotImplementedError()
    ##--your-code-ends-here--##
    
    return l, inliers, sample_count


In [ ]:
# Visible tests

# Load and run the algorithm
data = np.load(data_dir+'points.npy')
x, y = data[0,:], data[1,:]
m = len(x)
l, inliers, sample_count = ransac_line_fit(x, y)

# Existence, type and shape
assert isinstance(l, np.ndarray) and l.shape == (3,), "Return 'l' must be a NumPy array of shape (3,)."
assert hasattr(inliers, "__len__"), "'inliers' must be an index array or boolean mask."

# Inlier count
inlier_ratio = len(inliers) / m if not isinstance(inliers, np.ndarray) or inliers.dtype != bool else np.count_nonzero(inliers) / m
assert inlier_ratio > 0.6 and inlier_ratio < 0.8, "Inlier ratio is not in the range."

print("All visible tests passed.")

In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


### 2.4 Display RANSAC fit and inliers

The fitted line and the inliers will be plotted in this step. However, the code is currently missing the line parameters in a form suitable for plotting.

#### Task:
Implement `line_to_slope_intercept` function which transforms the output of the `ransac_line_fit` function into **slope–intercept form** so that it can be used for plotting. In slope–intercept form, the line is written as:
$
y = kx + b,
$
where $k$ and $b$ are the slope and intercept, respectively.

Derive the values of $k$ and $b$ based on the output of `ransac_line_fit`, which returns the line parameters in the form $ ax + by - d = 0 $.


In [ ]:
def line_to_slope_intercept(l):
    ##--your-code-starts-here--##
    # YOUR CODE HERE
    raise NotImplementedError()
    ##--your-code-ends-here--#  

    return k, b

# Load and plot the data points
data = np.load(data_dir+'points.npy')
x, y = data[0,:], data[1,:]
plt.plot(x, y, 'kx')

# Run the RANSAC function
l, inliers, s_count = ransac_line_fit(x, y)
# Transform the line coefficients to slope-intercept form
k, b = line_to_slope_intercept(l)

# Plot the resulting line and the inliers
x_line = np.arange(1, 101)
y_line = k * x_line + b
plt.plot(x_line, y_line, 'm-', label='RANSAC fit')
plt.plot(x[inliers], y[inliers], 'rx', markersize=6, label='Inliers')

plt.legend()
plt.title('Point distribution')
plt.axis('equal')
plt.show()


In [ ]:
# Visible tests
k, b = line_to_slope_intercept(l)
assert 'k' in globals(), "Variable 'k' (slope) was not defined."
assert 'b' in globals(), "Variable 'b' (intercept) was not defined."

# Size / shape checks
assert np.isscalar(k) or (np.ndim(k) == 0), "'k' must be a scalar."
assert np.isscalar(b) or (np.ndim(b) == 0), "'b' must be a scalar."

print("All visible tests passed.")

In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


In [ ]:
# HIDDEN TEST CELL
# This cell contains hidden test cases that will be evaluated after the deadline.
# Please do not remove or modify this cell, as it is required for grading.


## DEMO - Line detection by Hough transform (no points given)

Run the example cell below to illustrate line detection by Hough transform using OpenCV's built-in functions.

This demo performs the following steps:

- Detects **Canny edges** for the input image  
- Calculates the **Hough transform** on the Canny edge image  
- Displays the **Hough votes** in an accumulator array  
- Draws the **detected lines** on the image 

In [ ]:
# DEMO CELL
# Logistic sigmoid function
def sigm(x):
    return 1 / (1 + np.exp(-x))

# Read image
I = cv2.imread(data_dir+'board.png', 0)
r, c = I.shape

plt.figure(1)
plt.imshow(I, cmap='bone')
plt.title('Original image')
plt.axis('off')
# Find Canny edges. The input image for cv2.HoughLines should be
# a binary image, so a Canny edge image will do just fine.
# The Canny edge detector uses hysteresis thresholding, where
# there are two different threshold levels.
edges = cv2.Canny(I, 80, 130)
plt.figure(2)
plt.imshow(edges, cmap='gray')
plt.title('Canny edges')
plt.axis('off')
# Compute the Hough transform for the binary image returned by cv2.Canny
# cv2.HoughLines returns 2-element vectors containing (rho, theta)
# cv2.HoughLines(input image, radius resolution(pixels), angular resolution (radians),treshold )
H = cv2.HoughLines(edges, 0.5, np.pi/180, 5)

# Display the transform
theta = H[:,0,1].ravel()
rho = H[:,0,0].ravel()

# Create an acculumator array and the bin coordinates for voting
x_coord = np.arange(0, np.pi, np.pi/180)
y_coord = np.arange(np.amin(rho), np.amax(rho)+1, (np.amax(rho)+1)/50)

acc = np.zeros([np.size(y_coord),np.size(x_coord)])

# Perform the voting
for i in range(np.size(theta)):
    x_id = np.argmin(np.abs(x_coord-theta[i]))
    y_id = np.argmin(np.abs(y_coord-rho[i]))
    acc[y_id, x_id] +=  1

# Pass the values through a logistic sigmoid function and normalize
# (only for the purpose of better visualization)
#acc = sigm(acc)
acc /= np.amax(acc)

plt.figure(3)
plt.imshow(acc,cmap='bone')
plt.axis('off')

plt.title('Hough transform space')

plt.figure(4)
plt.imshow(acc,cmap='bone')
plt.axis('off')

plt.title('Hough transform space')

# Compute the Hough transform with higher threshold 
# for displaying ~30 strongest peaks in the transform space
H2 = cv2.HoughLines(edges, 1, np.pi/180, 150)

x2 = H2[:,:,1].ravel()
y2 = H2[:,:,0].ravel()

# Superimpose a plot on the image of the transform that identifies the peaks
plt.figure(4)
for i in range(np.size(x2)):
    x_id = np.argmin(abs(x_coord-x2[i]))
    y_id = np.argmin(abs(y_coord-y2[i]))
    plt.plot(x_id, y_id, 'xr','Linewidth',0.1)
    
# Visualize detected lines on top of the Canny edges.
plt.figure(5)
plt.imshow(edges, cmap='bone')
plt.title('Detected lines')
plt.axis('off')


for ind in range(0,len(H2)):
    line=H2[ind,0,:]
    rho=line[0]
    theta=line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    plt.plot((x1,x2),(y1,y2))
    
#plt.plot(xk, yk, 'm-')
plt.xlim([0,np.size(I,1)])
plt.ylim([0,np.size(I,0)])
plt.gca().invert_yaxis()
plt.show()